<a href="https://colab.research.google.com/github/rikdantas/Algoritmos-Estruturas-Dados-II/blob/main/week04/week04_U1T4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Desafio: Construindo Consultas Rápidas em um CSV**
### Aluno: Paulo Ricardo Dantas
### Matrícula: 20230000789

### Objetivo:
**Este projeto tem como objetivo estender o projeto guiado da Dataquest "Building Fast Queries on a CSV".**
### Etapas:
- Funcionalidades Adicionais:

- Análise de Complexidade:

- Documentação:


Note que o código utilizado, como já mencionado no README, vai ser reaproveitado do projeto guiado que foi feito na plataforma Dataquest, porém vão ser deixadas apenas as partes principais: A leitura do arquivo de dados (DataSet) usando o método open e a biblioteca csv; A última versão da classe Ïnventory" com os métodos implementados.

In [32]:
# Importando bibliotecas
import csv
import pprint
import re

In [33]:
with open("./laptops.csv", 'r') as file:
    reader = csv.reader(file)
    rows = list(reader)
    header = rows[0]
    rows = rows[1:]

In [34]:
def row_price(row):
    return row[-1]

class Inventory():

    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]

        for row in self.rows:
            row[-1] = int(row[-1])

        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row

        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])

        self.rows_by_price = sorted(self.rows, key=row_price)

    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None

    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None

    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row2 in self.rows:
            for row3 in self.rows:
                if row2[-1] + row3[-1] == dollars:
                    return True
        return False

    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False

    def find_laptop_with_price(self, target_price):
        range_start = 0
        range_end = len(self.rows_by_price) - 1
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2
            value = self.rows_by_price[range_middle][-1]
            if value == target_price:
                return range_middle
            elif value < target_price:
                range_start = range_middle + 1
            else:
                range_end = range_middle - 1
        if self.rows_by_price[range_start][-1] != target_price:
            return -1
        return range_start

    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0
        range_end = len(self.rows_by_price) - 1
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
        if self.rows_by_price[range_start][-1] <= target_price:
            return -1
        return range_start

    def find_laptops_in_price_range(self, min_price, max_price):
        laptops_in_range = []

        for row in self.rows:
            price = row[-1]
            if min_price <= price <= max_price:
                laptop_info = {}
                for i in range(len(self.header)):
                    laptop_info[self.header[i]] = row[i]
                laptops_in_range.append(laptop_info)

        return laptops_in_range

    def find_cheapest_laptop_with_specifications(self, capacity):
        # Use expressões regulares para extrair o tamanho da RAM e a capacidade de armazenamento
        match = re.match(r'(\d+)\s*GB\s+(\d+)\s*([A-Za-z\s]+)', capacity)
        if match:
            ram_size = int(match.group(1))
            storage_size = int(match.group(2))
            storage_type = match.group(3).strip()
        else:
            print("Capacidade inválida")
            return None

        # Agora você pode percorrer os dados da instância da classe e encontrar o laptop mais barato com as especificações dadas
        cheapest_laptop = None
        cheapest_price = float('inf')  # Inicialize com um valor alto

        for laptop in self.rows:
            # Verifique se a RAM e a capacidade de armazenamento estão presentes nas especificações do laptop
            if str(ram_size) in laptop[7] and str(storage_size) in laptop[8] and storage_type.lower() in laptop[8].lower():
                price = laptop[-1]
                if price < cheapest_price:
                    cheapest_laptop = laptop
                    cheapest_price = price

        if cheapest_laptop:
            return cheapest_laptop
        else:
            print(f"Nenhum laptop encontrado com as especificações: {capacity}")
            return None

        return cheapest_laptop

## [1/3] Funcionalidades Adicionais
- Implemente as seguintes funcionalidades adicionais:
  1. Consulta por faixa de preço (`min_price` e `max_price`).
  2. Consulta para encontrar o laptop mais barato com características específicas (ex: 8GB de RAM e HD de 256GB).

OBS: Para fazer com que as novas features funcionem no nosso código, elas precisam ser adicionadas dentro das classes, então aqui vou deixar apenas destacado os códigos implementados.

**Código para consultar por faixa de preço:**

```
def find_laptops_in_price_range(self, min_price, max_price):
    laptops_in_range = []

    for row in self.rows:
        price = row[-1]
        if min_price <= price <= max_price:
            laptop_info = {}
            for i in range(len(self.header)):
                laptop_info[self.header[i]] = row[i]
            laptops_in_range.append(laptop_info)

    return laptops_in_range
```



In [35]:
# Testando a função find_laptops_in_price_range
inventory = Inventory("laptops.csv")
# Para melhor visualização, vai ser utilizado a biblioteca pprint, para imprimir em coluna ao invés de linha
pp = pprint.PrettyPrinter()
pp.pprint(inventory.find_laptops_in_price_range(1100, 1110))

[{'Company': 'HP',
  'Cpu': 'Intel Core i7 8550U 1.8GHz',
  'Gpu': 'Intel UHD Graphics 620',
  'Id': '8036099',
  'Inches': '13.3',
  'Memory': '512GB SSD',
  'OpSys': 'Windows 10',
  'Price': 1103,
  'Product': 'ProBook 430',
  'Ram': '8GB',
  'ScreenResolution': 'Full HD 1920x1080',
  'TypeName': 'Notebook',
  'Weight': '1.49kg'},
 {'Company': 'Lenovo',
  'Cpu': 'Intel Core i7 7700HQ 2.8GHz',
  'Gpu': 'Nvidia GeForce GTX 1050M',
  'Id': '7157825',
  'Inches': '15.6',
  'Memory': '128GB SSD +  1TB HDD',
  'OpSys': 'Windows 10',
  'Price': 1109,
  'Product': 'Legion Y520-15IKBN',
  'Ram': '8GB',
  'ScreenResolution': 'IPS Panel Full HD 1920x1080',
  'TypeName': 'Gaming',
  'Weight': '2.5kg'},
 {'Company': 'Toshiba',
  'Cpu': 'Intel Core i5 6200U 2.3GHz',
  'Gpu': 'Intel HD Graphics 520',
  'Id': '5342965',
  'Inches': '14',
  'Memory': '128GB SSD',
  'OpSys': 'Windows 10',
  'Price': 1105,
  'Product': 'Tecra Z40-C-12X',
  'Ram': '4GB',
  'ScreenResolution': 'IPS Panel Full HD 1920x108

In [36]:
# Exemplo quando o a função não acha nenhum notebook na faixa de preço passada como parametro
pp.pprint(inventory.find_laptops_in_price_range(100, 170))

[]


**Código para consultar o notebook mais barato, dada as especificações:**

```
def find_cheapest_laptop_with_specifications(self, capacity):
        match = re.match(r'(\d+)\s*GB\s+(\d+)\s*([A-Za-z\s]+)', capacity)
        if match:
            ram_size = int(match.group(1))
            storage_size = int(match.group(2))
            storage_type = match.group(3).strip()
        else:
            print("Capacidade inválida")
            return None

        cheapest_laptop = None
        cheapest_price = float('inf')  # Inicialize com um valor alto

        for laptop in self.rows:
            if str(ram_size) in laptop[7] and str(storage_size) in laptop[8] and storage_type.lower() in laptop[8].lower():
                price = laptop[-1]
                if price < cheapest_price:
                    cheapest_laptop = laptop
                    cheapest_price = price

        if cheapest_laptop:
            return cheapest_laptop
        else:
            print(f"Nenhum laptop encontrado com as especificações: {capacity}")
            return None

        return cheapest_laptop
```


In [37]:
# Testando a função find_cheapest_laptop_with_specifications
# Para melhor visualização, vai ser utilizado a biblioteca pprint, para imprimir em coluna ao invés de linha
pp.pprint(inventory.find_cheapest_laptop_with_specifications("16 GB 1 TB SSD"))

['2666332',
 'Lenovo',
 'Yoga 900-13ISK',
 '2 in 1 Convertible',
 '13.3',
 'IPS Panel Quad HD+ / Touchscreen 3200x1800',
 'Intel Core i7 6560U 2.2GHz',
 '16GB',
 '1TB SSD',
 'Intel Iris Graphics 540',
 'Windows 10',
 '1.3kg',
 1799]


In [38]:
# Exemplo quando o a função não acha nenhum notebook com as especificações passadas como parametro
pp.pprint(inventory.find_cheapest_laptop_with_specifications("10 GB 1 TB SSD"))

Nenhum laptop encontrado com as especificações: 10 GB 1 TB SSD
None


## [2/3] Análise de Complexidade
- Realize a análise de complexidade para as funcionalidades implementadas, focando em Big O, Big θ, e Big Ω. Documente suas descobertas no arquivo README do repositório.

## [3/3] Documentação
- Adicione comentários e documentação ao código para explicar o que cada parte faz.